In [5]:
%pip install --upgrade --pre torch torchvision --extra-index-url https://download.pytorch.org/whl/nightly/cu118

%pip install --upgrade diffusers transformers accelerate

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cu118
  ERROR: HTTP error 403 while getting https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from https://download.pytorch.org/whl/nightly/cu118/nvidia-cuda-nvrtc-cu12/)
ERROR: Could not install requirement nvidia-cuda-nvrtc-cu12==12.1.105 from https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from torch) because of HTTP error 403 Client Error: Forbidden for url: https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl for URL https://download.pytorch.org/whl/nightly/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from https://download.pytorch.org/whl/nightly/cu118/nvidia-cuda-nvrtc-cu12/)
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 86.8 MB/s eta 0:00:0

In [ ]:
import torch
from PIL import Image
from diffusers import StableVideoDiffusionPipeline
from src.unet_spatio_temporal_condition import UNetSpatioTemporalConditionModel
from diffusers.utils import load_image, export_to_video, export_to_gif
from pathlib import Path
import argparse

RuntimeError: Failed to import diffusers.loaders.unet because of the following error (look up to see its traceback):
module 'torch' has no attribute 'float8_e4m3fn'

In [ ]:
# 1) Lade dein fine-getuntes UNet aus deinem Output-Ordner
finetuned_unet = UNetSpatioTemporalConditionModel.from_pretrained(
    "https://playground9377540152.blob.core.windows.net/2e454750-288f-4df2-ae37-4ef303fd2d74-34z2jabu5s276j98xep7l5p30z/SVDTraining/SVD_Xtend/OutputTrainedModel",
    torch_dtype=torch.bfloat16,                 # wenn du bf16 trainiert hast
    low_cpu_mem_usage=False,
)

# 2) Baue die Pipeline mit dem Original-Checkpoint, injiziere dein UNet
pipe = StableVideoDiffusionPipeline.from_pretrained(
    "/mnt/models/stable-video-diffusion-img2vid-xt",
    unet=finetuned_unet,
    torch_dtype=torch.bfloat16,
    local_files_only=True,
)

# 3) Caste alle Module auf BF16 und ein contiguous Layout
pipe.unet = pipe.unet.to(torch.bfloat16, memory_format=torch.contiguous_format).to("cuda")
pipe.vae  = pipe.vae.to(torch.bfloat16, memory_format=torch.contiguous_format).to("cuda")
pipe.image_encoder = pipe.image_encoder.to(torch.bfloat16, memory_format=torch.contiguous_format).to("cuda")

In [ ]:
# 4) Inference
script_dir = Path(__file__).parent
demo_path  = script_dir / "demo.png"
image = load_image(str(demo_path)).resize((768, 768))
generator = torch.manual_seed(42)

with torch.inference_mode():
    output = pipe(
        image,
        num_frames=10,
        width=768,
        height=768,
        decode_chunk_size=8,
        motion_bucket_id=127,
        fps=7,
        num_inference_steps=30,
        generator=generator,
    )
    frames = output.frames[0]

# 5) Speichern (als GIF oder Video)
export_to_gif(frames, "output.gif", fps=7)
# oder
export_to_video(frames, "output.mp4", fps=7)